Create model using SVM 

In [2]:
import pandas as pd

In [13]:
train_data = pd.read_csv('train-data.csv', index_col=[0])
test_data = pd.read_csv('test-data.csv', index_col=[0])

train_data.head()

y_train = train_data.target
y_test = test_data.target

X_train = train_data.drop(['target', 'target_name'], axis='columns')
X_test = test_data.drop(['target', 'target_name'], axis='columns')

In [18]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
clf = model.fit(X_train, y_train)

In [15]:
model.score(X_test, y_test) 

0.8336950838891617

In [ ]:
# use stratified k-fold as validation method
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4)
results = cross_val_score(clf, X_train, y_train, cv=skf)
print("Accuracy: %.2f%%" % (results.mean()*100.0))